In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [2]:
# Scale the pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# Convert the labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
# Reshape
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
# Split data into 90% training and 10% validation sets
split = int(0.9 * len(x_train))
x_train, x_val = x_train[:split], x_train[split:]
y_train, y_val = y_train[:split], y_train[split:]

In [4]:
# Define LeNet model architecture
model = keras.Sequential([
    keras.layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(units=120, activation='relu'),
    keras.layers.Dense(units=84, activation='relu'),
    keras.layers.Dense(units=10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Define tensorboard logs directory
log_dir="C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionLeNet"
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)

In [6]:
# Train the model
model.fit(x_train, y_train, epochs=30, batch_size=128, validation_data=(x_val, y_val), callbacks=[tensorboard])

Epoch 1/30
422/422 [==============================] - 23s 48ms/step - loss: 0.6926 - accuracy: 0.7560 - val_loss: 0.4933 - val_accuracy: 0.8267
Epoch 2/30
422/422 [==============================] - 19s 44ms/step - loss: 0.4408 - accuracy: 0.8415 - val_loss: 0.4090 - val_accuracy: 0.8498
Epoch 3/30
422/422 [==============================] - 21s 49ms/step - loss: 0.3844 - accuracy: 0.8623 - val_loss: 0.3926 - val_accuracy: 0.8612
Epoch 4/30
422/422 [==============================] - 24s 57ms/step - loss: 0.3495 - accuracy: 0.8722 - val_loss: 0.3396 - val_accuracy: 0.8777
Epoch 5/30
422/422 [==============================] - 20s 47ms/step - loss: 0.3253 - accuracy: 0.8810 - val_loss: 0.3256 - val_accuracy: 0.8807
Epoch 6/30
422/422 [==============================] - 31s 74ms/step - loss: 0.3091 - accuracy: 0.8877 - val_loss: 0.3312 - val_accuracy: 0.8765
Epoch 7/30
422/422 [==============================] - 39s 93ms/step - loss: 0.2895 - accuracy: 0.8929 - val_loss: 0.2989 - val_accuracy:

In [7]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)

# Generate predictions on the test set
y_pred = model.predict(x_test)

# Convert predicted probabilities to predicted class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert true labels to one-hot encoding
y_true = np.argmax(y_test, axis=1)

313/313 [==============================] - 5s 11ms/step


In [8]:
# Compute evaluation metrics
conf_matrix = confusion_matrix(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1score = f1_score(y_true, y_pred_classes, average='macro')
auc_roc = roc_auc_score(y_test, y_pred, multi_class='ovr')

# Print the evaluation metrics
print("Test Loss:", test_loss)
print("Accuracy:", test_acc)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)
print("AU-ROC:", auc_roc)

Test Loss: 0.32006314396858215
Accuracy: 0.902899980545044
Confusion Matrix:
 [[820   0  26  36   5   1  97   0  15   0]
 [  0 976   1  15   3   0   3   0   2   0]
 [ 14   3 869  10  60   0  41   0   3   0]
 [  4   4  12 933  31   0  14   0   2   0]
 [  0   0  63  32 864   0  41   0   0   0]
 [  0   0   0   2   0 967   0  20   1  10]
 [ 83   0  82  45  95   0 681   0  14   0]
 [  0   0   0   0   0  13   0 971   0  16]
 [  1   0   4   5   2   2   2   2 981   1]
 [  1   0   0   0   0   4   0  28   0 967]]
Precision: 0.9026010679309644
Recall: 0.9029
F1-score: 0.9020184987635108
AU-ROC: 0.9927512833333333


In [9]:
# Log evaluation metrics to TensorBoard
test_writer = tf.summary.create_file_writer(log_dir)
with test_writer.as_default():
    tf.summary.scalar('Accuracy', test_acc, step=0)
    tf.summary.scalar('Precision', precision, step=0)
    tf.summary.scalar('Recall', recall, step=0)
    tf.summary.scalar('F1-score', f1score, step=0)
    tf.summary.scalar('AU-ROC', auc_roc, step=0)

In [10]:
# tensorboard --logdir_spec=FashionMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionMLP",FashionLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionLeNet",MnistLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistLeNet",MnistMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistMLP"